### TODO: General-purpose sensitivity computation 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

census = pd.read_csv('drive/MyDrive/adult.csv')
census.head(3)

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K


### Exercise 1: Calculate a DP-count of individuals of 'age' > 40
Calculate the sensitivity and experiment with different epsilon values

In [ ]:
def add_laplace_noise(s, sensitivity, epsilon):
  return s + np.random.laplace(loc=0, scale=sensitivity/epsilon)

def percentage_error(orig, est):
  return ((orig-est) / orig) * 100

#print('Original count:', len(census.query('age > 40')))
orig_count = len(census.query('age > 40'))
dp_count = add_laplace_noise(len(census.query('age > 40')), 1, 0.01)
print(orig_count, dp_count, percentage_error(orig_count, dp_count))

13443 13449.815972273389 -0.05070276183432998


### Exercise 2: Calculate a DP-sum of the attribute 'fnlwgt'
Calculate the sensitivity and experiment with different epsilon values.

In [ ]:
orig_sum = census['fnlwgt'].sum()
dp_sum = add_laplace_noise(census['fnlwgt'].sum(), census['fnlwgt'].max(), 0.01)
print(orig_sum, dp_sum, percentage_error(orig_sum, dp_sum))

6179373392 6107062957.503553 1.170190404581504


### Exercise 3: Calculate a DP-mean of working hours for people with income >50K.

In [ ]:
filtered_wh = census.query('income == ">50K"')['hours.per.week']
orig_mean = filtered_wh.mean()
dp_mean = add_laplace_noise(filtered_wh.sum(), filtered_wh.max(), 0.01) / add_laplace_noise(len(filtered_wh), 1, 0.01)
print(orig_mean, dp_mean, percentage_error(orig_mean, dp_mean))

45.473026399693914 47.513726425240776 -4.487715437300645


### Exercise 4: Use PyDP to compare with the results obtained.

In [ ]:
! pip install python-dp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pydp as dp
from pydp.algorithms.laplacian import BoundedSum, BoundedMean, Count

In [ ]:
x = BoundedSum(epsilon=0.2, delta=0, dtype="float")

pydp_sum = x.quick_result(list(census['fnlwgt']))
print(orig_sum, pydp_sum, percentage_error(orig_sum, pydp_sum))

6179373392 6097908184.303986 1.3183409146545777


In [ ]:
x = Count(0.01)
pydp_count = x.quick_result(list(census.query('age > 40')['age']))
print(orig_count, pydp_count, percentage_error(orig_count, pydp_count))

13443 13392 0.3793796027672395


In [ ]:
x = BoundedMean(0.1)
pydp_mean = x.quick_result(list(filtered_wh))
print(orig_mean, pydp_mean, percentage_error(orig_mean, pydp_mean))

45.473026399693914 45.49680485642466 -0.05229134415145459
